In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


s=Service(ChromeDriverManager().install())




====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/97.0.4692.71/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\riccie\.wdm\drivers\chromedriver\win32\97.0.4692.71]


In [9]:
from logger import *    

logger = get_logger(path = 'test_log_file.log')


In [10]:
logger.info('START')
driver = webdriver.Chrome(service=s)
url = 'https://www.istat.it/it/metodi-e-strumenti/web-service-sdmx#Aggiornamentodeidataseterelativequery-2'
driver.get(url )

try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "rsssep"))
    )
    page_source = driver.page_source

finally:
    driver.quit()


[2022-01-08 17:57:16,957]:[sdmx_aggiornamento]:[<module>]:[INFO]: START
[2022-01-08 17:57:16,957]:[sdmx_aggiornamento]:[<module>]:[INFO]: START


In [11]:
from bs4 import BeautifulSoup
import lxml
soup = BeautifulSoup(page_source,'html.parser')


In [184]:
#Common
def Get_box_info(box):
    
    titolo=box.find(class_='title').text
    
    data=box.find(class_='date').text
    
    text=box.find('textarea').text
    start=start_regex.findall(text)
    end=end_regex.findall(text)
    ID=id_regex.findall(text)
    
    return dict(title=titolo,
                date=data,
                start=start,
                end=end,
                ID=ID)




______________ box numero 1 _______________
{'title': 'Conto trimestrale', 'date': '07/01/2022 10:14', 'start': ['1999'], 'end': ['2021'], 'ID': ['95_42']}
______________ box numero 2 _______________
{'title': 'Nic - mensili dal 2016 (base 2015)', 'date': '05/01/2022 11:11', 'start': ['2021'], 'end': ['2021'], 'ID': ['167_744']}
______________ box numero 3 _______________
{'title': 'Ipca - mensili dal 2001 (base 2015)', 'date': '05/01/2022 11:13', 'start': ['2021'], 'end': ['2021'], 'ID': ['168_760']}
______________ box numero 4 _______________
{'title': 'Istituti di cura  - comuni', 'date': '03/01/2022 08:31', 'start': ['2018'], 'end': ['2019'], 'ID': ['43_994']}


In [200]:
#Common
def Get_box_info(box):
    
    titolo=box.find(class_='title').text
    
    data=box.find(class_='date').text
    
    text=box.find('textarea').text
    start=start_regex.findall(text)
    end=end_regex.findall(text)
    period=start,end
    ID=id_regex.findall(text)
    
    return dict(flow=titolo,
                data_aggiornamento=data,
                periodo_aggiornamento=period,
                id=ID)
 
page=soup.find_all(class_=cl)
i=1
for box in page:
        print('______________ box numero', i, '_______________')
        Info=Get_box_info(box)
        print(Info)
        Monitor.create(**Info)
        i=i+1


______________ box numero 1 _______________
{'flow': 'Conto trimestrale', 'data_aggiornamento': '07/01/2022 10:14', 'periodo_aggiornamento': (['1999'], ['2021']), 'id': ['95_42']}


TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [188]:
###aggiunta gestione casistica unica pagina di aggiornamenti
try:
    l=soup.find(class_='simplePagerNav')
    result=[]
    ndt=0
    len(l)
    logger.debug('Casistica multi pagina')
    for i in list(range(1, len(l)+1)):
        cl='simplePagerPage'+str(i)
        page=soup.find_all(class_=cl)
        for box in page:
                    print('______________ box numero', i, '_______________')
                    Info=Get_box_info(box)
                    print(Info['title'])
                    print('aggiornato il ' + info['date'])
                    print('info per query -> ',  info['ID'],  info['start'], info['end'])
                    print(' ')
                    i=i+1
                    ndt=ndt+1 
    logger.info(f'Identificate {len(l)} pagine di aggiornamenti per un totale di {len(result)} dataset')
except TypeError as e :   
    logger.error(e)
    logger.debug('Except - casistica unica pagina')
    cl='simplePagerPage1'
    page=soup.find_all(class_=cl)
    i=1
    for box in page:
        print('______________ box numero', i, '_______________')
        Info=Get_box_info(box)
        print(Info['title'])
        print('aggiornato il ' + Info['date'])
        print('info per query -> ',  Info['ID'],  Info['start'], Info['end'])
        print(' ')
        i=i+1

    logger.info(f'Identificata 1 pagina di aggiornamenti per un totale di {len(page)} dataset ')
      



[2022-01-08 20:22:30,094]:[sdmx_aggiornamento]:[<module>]:[ERROR]: object of type 'NoneType' has no len()
[2022-01-08 20:22:30,094]:[sdmx_aggiornamento]:[<module>]:[ERROR]: object of type 'NoneType' has no len()
[2022-01-08 20:22:30,100]:[sdmx_aggiornamento]:[<module>]:[INFO]: Identificata 1 pagina di aggiornamenti per un totale di 4 dataset 
[2022-01-08 20:22:30,100]:[sdmx_aggiornamento]:[<module>]:[INFO]: Identificata 1 pagina di aggiornamenti per un totale di 4 dataset 


______________ box numero 1 _______________
Conto trimestrale
aggiornato il 07/01/2022 10:14
info per query ->  ['95_42'] ['1999'] ['2021']
 
______________ box numero 2 _______________
Nic - mensili dal 2016 (base 2015)
aggiornato il 05/01/2022 11:11
info per query ->  ['167_744'] ['2021'] ['2021']
 
______________ box numero 3 _______________
Ipca - mensili dal 2001 (base 2015)
aggiornato il 05/01/2022 11:13
info per query ->  ['168_760'] ['2021'] ['2021']
 
______________ box numero 4 _______________
Istituti di cura  - comuni
aggiornato il 03/01/2022 08:31
info per query ->  ['43_994'] ['2018'] ['2019']
 


In [7]:
valori={}
temp={}
keys=['flow','data_aggiornamento','note']
lista=[]
logger.info('Inserisco info in tabella monitor')
for item in result:
    for idx, val in enumerate(item):
        
        temp[keys[idx]]=(val.text)
        valori=temp.copy()
    lista.append(valori)


[2022-01-07 10:57:22,745]:[sdmx_aggiornamento]:[<module>]:[INFO]: Inserisco info in tabella monitor


In [55]:
from peewee import *

db = SqliteDatabase('monitor_dev.db')

import datetime


class BaseModel(Model):
    class Meta:
        database = db


class Monitor(BaseModel):
    id = BigAutoField(primary_key = True)
    data_run = DateTimeField(default=datetime.datetime.now)
    flow = TextField(null=True)
    aggiornato = BooleanField(default=True)
    data_aggiornamento=DateTimeField(formats='%d/%m/%Y %H:%M', null=True, index=True)
    download  =  BooleanField(default=False)
    #volume= DoubleField(null=True)
    note = TextField(null=True)

    

def initialize_db():
    db.connect()
    db.create_tables([Monitor], safe = True)
    db.close()    

    
initialize_db()

In [201]:
for row in rows:
    print(row.flow)

Tensione contrattuale
Clima di fiducia delle imprese
Fiducia dei consumatori
Retribuzioni orarie  dei dipendenti del settore privato
Retribuzioni contrattuali per settore economico (Ateco)
Retribuzioni contrattuali per tipo di contratto
Fiducia delle imprese del commercio
Fiducia delle imprese servizi
Fiducia delle imprese costruzioni
Fiducia delle imprese manifatturiere
Protesti
Ore lavorate - Imprese con almeno 500 dipendenti
Sequenza dei conti (milioni di euro)
Prodotto interno lordo lato produzione (milioni di euro)
Occupazione regolare ed irregolare
Produzione e valore aggiunto, Redditi da lavoro, Imposte e contributi alla produzione, Spesa per consumi finali famiglie, Investimenti fissi lordi, Stock di capitale e ammortamenti, Valori pro capite
Occupazione regolare e irregolare
Posizioni lavorative - Imprese con almeno 500 dipendenti
Retribuzioni lorde - Imprese con almeno 500 dipendenti
Imprese con almeno 500 dipendenti
Tavole  di mortalità
Indice della produzione nelle costruzi

In [10]:
n=0
for item in lista:
    try:
        n=n+1
        logger.info(f'inserisco riga {n}')
        Monitor.create(**item)
            
    except Exception as e:

        logger.error(e)


In [3]:
db.get_tables()


NameError: name 'db' is not defined

In [2]:

rows=Monitor.select()
for row in rows:
    print(f' Il data-set {row.flow} è stato aggiornato il {row.data_aggiornamento.split()[0]} alle {row.data_aggiornamento.split()[1]}')

NameError: name 'Monitor' is not defined

In [137]:

# Python program to read
# json file
 
 
import json
 
# Opening JSON file
f = open('collection.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
for i in data['item']:
    for j in i:
        print(j['name'])
 
# Closing file
f.close()    


TypeError: string indices must be integers

In [13]:
######################## REGEX ##############################
import re
rows=Monitor.select()
start_regex=re.compile('(?<=<query:StartTime>)(.*)(?=</query:StartTime>)')
end_regex=re.compile('(?<=<query:EndTime>)(.*)(?=</query:EndTime>)')
id_regex=re.compile('(?<=<ID>)(.*)(?=</ID>)')
start=start_regex.findall(rows[45].note)
end=end_regex.findall(rows[45].note)
ID=id_regex.findall(rows[45].note)

start,end,ID

(['2000'], ['2021'], ['111_263'])

In [2]:
############### PANDAS
import pandasdmx as sdmx

ISTAT = sdmx.Request('ISTAT')

flow_msg = ISTAT.dataflow()


C:\Users\riccie\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandasdmx\remote.py:11: RuntimeWarning: optional dependency requests_cache is not installed; cache options to Session() have no effect
  warn(


In [15]:
dataflows=sdmx.to_pandas(flow_msg.dataflow)
dataflows[dataflows.str.contains('pop', case=False)] #### search by string name

popres_msg = ISTAT.dataflow('22_289')
print(popres_msg.dataflow)


DictLike([('22_289', <DataflowDefinition IT1:22_289(1.4): Resident population  on 1st January>)])


In [7]:
popres_msg = ISTAT.dataflow('22_315')


In [8]:
ISTAT.datastructure('DCIS_POPORESBIL1')

<pandasdmx.StructureMessage>
  <Header>
    id: 'IDREF372'
    prepared: '2022-01-10T14:56:08.837279+01:00'
    receiver: <Agency Unknown>
    sender: <Agency Unknown>
    source: 
    test: False
  response: <Response [200]>
  Codelist (9): CL_BASE_YEAR CL_FLAG CL_FREQ CL_ITTER107 CL_OBS_CONF CL...
  ConceptScheme (2): CROSS_DOMAIN VARIAB_ALL
  DataflowDefinition (1): 22_315
  DataStructureDefinition (1): DCIS_POPORESBIL1

In [ ]:
measures=popres_msg.structure.DCIS_POPRES1.measures
attr=popres_msg.structure.DCIS_POPRES1.attributes
dim=popres_msg.structure.DCIS_POPRES1.dimensions
FREQ=dim.components[0]
popres_msg.codelist.CL_SEXISTAT1.items
comp=attr.components[8]
popres_msg.concept_scheme

popres_msg

In [ ]:
dsd=popres_msg.structure.DCIS_POPRES1
dsd.attributes
local=dsd.dimensions.get('FREQ').local_representation

popres_msg.codelist.CL_ETA1.items['Y64']

popres_msg.constraint.CONSTR_22_289


In [68]:
###CONSTRAINT

msg=ISTAT.dataflow('22_289')
msg.constraint.CONSTR_22_289.data_content_region[0].member.keys()


dict_keys([<Dimension ITTER107>, <Dimension TIPO_INDDEM>, <Dimension SESSO>, <Dimension ETA>, <Dimension STATO_CIV>, <Dimension FREQ>])

In [4]:
msg=ISTAT.dataflow('101_959')
#msg.constraint.CONSTR_22_289.data_content_region[0].member.keys()


In [18]:
ISTAT.dataflow('101_959')

<pandasdmx.StructureMessage>
  <Header>
    id: 'IDREF1665'
    prepared: '2022-01-11T11:31:27.895885+01:00'
    receiver: <Agency Unknown>
    sender: <Agency Unknown>
    source: 
    test: False
  response: <Response [200]>
  Categorisation (1): 101_959@IT1@10@ISTAT_DW@10@101
  CategoryScheme (1): ISTAT_DW
  Codelist (11): CL_BASE_YEAR CL_FLAG CL_FREQ CL_ITTER107 CL_NUMEROSITA...
  ConceptScheme (2): CROSS_DOMAIN VARIAB_ALL
  DataflowDefinition (1): 101_959
  DataStructureDefinition (1): DCSP_LATTE